# Handling Gaze Events

## What you will learn in this tutorial:

* how to detect different events using different algorithms like IDT, IVT and microsaccades
* how to compute event properties like peak velocity and amplitude
* how to save and load your event data


## Detecting Gaze Events
## Preparations
At first we import `pymovements` as the alias `pm` for convenience.

In [5]:
import pymovements as pm

Then we download a dataset `ToyDataset` and load its data:

In [12]:
dataset = pm.Dataset('ToyDataset', path='data/ToyDataset')
dataset.download()
dataset.load()

INFO:pymovements.dataset.dataset:        You are downloading the pymovements Toy Dataset. Please be aware that pymovements does not
        host or distribute any dataset resources and only provides a convenient interface to
        download the public dataset resources that were published by their respective authors.

        Please cite the referenced publication if you intend to use the dataset in your research.
        


Using already downloaded and verified file: data\ToyDataset\downloads\pymovements-toy-dataset.zip
Extracting pymovements-toy-dataset.zip to data\ToyDataset\raw


100%|██████████| 23/23 [00:00<00:00, 240.33it/s]


  0%|          | 0/20 [00:00<?, ?it/s]

text_id,page_id,filepath
i64,i64,str
0,1,"""aeye-lab-pymovements-toy-datas…"
0,2,"""aeye-lab-pymovements-toy-datas…"
0,3,"""aeye-lab-pymovements-toy-datas…"
0,4,"""aeye-lab-pymovements-toy-datas…"
0,5,"""aeye-lab-pymovements-toy-datas…"
…,…,…
3,1,"""aeye-lab-pymovements-toy-datas…"
3,2,"""aeye-lab-pymovements-toy-datas…"
3,3,"""aeye-lab-pymovements-toy-datas…"


The dataset consist of gaze data in 20 files (check `Dataset/gaze` above). Every `Gaze` has some samples with six columns (check `Gaze/samples`): [time, stimuli_x, stimuli_y, text_id, page_id, pixel]. The `Gaze/events` DataFrame is empty so far. To be able to calculate events, we need to do some basic preprocessing, which will add new columns to the dataset samples DataFrame:

* `pix2deg()`: adds `position` column with degrees from the screen center needed by the `idt` algorithm
* `pos2vel()`: adds `velocity` column with gaze velocities needed by `microsaccades` and `ivt` algorithms


In [13]:
dataset.pix2deg()
dataset.pos2vel('smooth')
dataset.gaze[0]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

time,stimuli_x,stimuli_y,text_id,page_id,pixel,position,velocity
i64,f64,f64,i64,i64,list[f64],list[f64],list[f64]
1988145,-1.0,-1.0,0,1,"[206.8, 152.4]","[-10.697598, -8.852399]","[null, null]"
1988146,-1.0,-1.0,0,1,"[206.9, 152.1]","[-10.695183, -8.859678]","[null, null]"
1988147,-1.0,-1.0,0,1,"[207.0, 151.8]","[-10.692768, -8.866956]","[1.610194, -5.256267]"
1988148,-1.0,-1.0,0,1,"[207.1, 151.7]","[-10.690352, -8.869381]","[0.402548, -4.447465]"
1988149,-1.0,-1.0,0,1,"[207.0, 151.5]","[-10.692768, -8.874233]","[0.402561, -3.234462]"
…,…,…,…,…,…,…,…
2005363,-1.0,-1.0,0,1,"[361.0, 415.4]","[-6.932438, -2.386672]","[-63.266374, -21.085616]"
2005364,-1.0,-1.0,0,1,"[358.0, 414.5]","[-7.006376, -2.408998]","[-63.249652, -19.431326]"
2005365,-1.0,-1.0,0,1,"[355.8, 413.8]","[-7.060582, -2.426362]","[-60.359624, -15.710061]"


Now every `Gaze/samples` DataFrame has two more columns: position and velocity which will be used by the event detection algorithms.

## Detecting Events

*pymovements* provides a range of event detection methods for several types of gaze events.

See the reference for [pymovements.events](https://pymovements.readthedocs.io/en/latest/reference/pymovements.events.html) to get an overview of all the supported methods.

For this tutorial we will use the I-DT and I-VT (`idt` and `ivt`) algorithms for detecting fixations and the `microsaccades` algorithm for detecting saccades.

Let's start with fixations detection using the `idt` algorithm with the `dispersion_threshold` equal to 2.7:

In [15]:
dataset.detect_events('idt', dispersion_threshold=2.7)

0it [00:00, ?it/s]

text_id,page_id,name,onset,offset,duration
i64,i64,str,i64,i64,i64
0,1,"""fixation""",1988145,1988563,418
0,1,"""fixation""",1988564,1988750,186
0,1,"""fixation""",1988751,1989178,427
0,1,"""fixation""",1989179,1989436,257
0,1,"""fixation""",1989437,1989600,163
…,…,…,…,…,…
0,1,"""fixation""",2003929,2004090,161
0,1,"""fixation""",2004091,2004363,272
0,1,"""fixation""",2004364,2004883,519


The detected events are added as rows with the name `fixation` to the event dataframe:

In [16]:
dataset.events[0]

text_id,page_id,name,onset,offset,duration
i64,i64,str,i64,i64,i64
0,1,"""fixation""",1988145,1988563,418
0,1,"""fixation""",1988564,1988750,186
0,1,"""fixation""",1988751,1989178,427
0,1,"""fixation""",1989179,1989436,257
0,1,"""fixation""",1989437,1989600,163
…,…,…,…,…,…
0,1,"""fixation""",2003929,2004090,161
0,1,"""fixation""",2004091,2004363,272
0,1,"""fixation""",2004364,2004883,519


As you can see, 56 fixations were found for the first file.

Now let's try another algorithm `ivt` with velocity_threshold=20. Because we don't want to mix fixations found by different algorithms we add `name` parameter with 'fixation.ivt'

In [17]:
dataset.detect_events('ivt', velocity_threshold=20, name='fixation.ivt')
dataset.events[0]

0it [00:00, ?it/s]

text_id,page_id,name,onset,offset,duration
i64,i64,str,i64,i64,i64
0,1,"""fixation""",1988145,1988563,418
0,1,"""fixation""",1988564,1988750,186
0,1,"""fixation""",1988751,1989178,427
0,1,"""fixation""",1989179,1989436,257
0,1,"""fixation""",1989437,1989600,163
…,…,…,…,…,…
0,1,"""fixation.ivt""",2004132,2004331,199
0,1,"""fixation.ivt""",2004399,2004687,288
0,1,"""fixation.ivt""",2004714,2004878,164


Now we have additional rows with name='fixations.ivt'.

Let's try to use the `microsaccades` algorithm to detect fixations.

In [18]:
dataset.detect_events('microsaccades', minimum_duration=12)

0it [00:00, ?it/s]

text_id,page_id,name,onset,offset,duration
i64,i64,str,i64,i64,i64
0,1,"""fixation""",1988145,1988563,418
0,1,"""fixation""",1988564,1988750,186
0,1,"""fixation""",1988751,1989178,427
0,1,"""fixation""",1989179,1989436,257
0,1,"""fixation""",1989437,1989600,163
…,…,…,…,…,…
0,1,"""saccade""",2004373,2004385,12
0,1,"""saccade""",2004688,2004704,16
0,1,"""saccade""",2004879,2004901,22


The detected events are added as rows with the name `saccade` to the event dataframe:

In [19]:
dataset.events[0]

text_id,page_id,name,onset,offset,duration
i64,i64,str,i64,i64,i64
0,1,"""fixation""",1988145,1988563,418
0,1,"""fixation""",1988564,1988750,186
0,1,"""fixation""",1988751,1989178,427
0,1,"""fixation""",1989179,1989436,257
0,1,"""fixation""",1989437,1989600,163
…,…,…,…,…,…
0,1,"""saccade""",2004373,2004385,12
0,1,"""saccade""",2004688,2004704,16
0,1,"""saccade""",2004879,2004901,22


Now there are three sets of events in the `dataset.events` DataFrame with different values in the 'name' column:

In [29]:
set(dataset.events[0].frame['name'])

{'fixation', 'fixation.ivt', 'saccade'}

## Computing Event Properties

*pymovements* provides a range of event properties.

See the reference for [pymovements.events](https://pymovements.readthedocs.io/en/latest/reference/pymovements.events.html) to get an overview of all the supported properties.

For this tutorial we will compute several properties of saccades.

We start out with the peak velocity:

In [30]:
dataset.compute_event_properties("peak_velocity")

dataset.events[0]

  0%|          | 0/20 [00:00<?, ?it/s]

text_id,page_id,name,onset,offset,duration,peak_velocity
i64,i64,str,i64,i64,i64,f64
0,1,"""fixation""",1988145,1988563,418,200.144558
0,1,"""fixation""",1988564,1988750,186,249.67823
0,1,"""fixation""",1988751,1989178,427,211.598748
0,1,"""fixation""",1989179,1989436,257,189.183243
0,1,"""fixation""",1989437,1989600,163,255.077509
…,…,…,…,…,…,…
0,1,"""saccade""",2004373,2004385,12,70.374183
0,1,"""saccade""",2004688,2004704,16,175.646379
0,1,"""saccade""",2004879,2004901,22,209.46361


Check above that a new column with the name `peak_velocity` has appeared in the event DataFrame.

We can also pass a list of properties. Let's add the amplitude and dispersion:

In [31]:
dataset.compute_event_properties(["amplitude", "dispersion"])

dataset.events[0]

  0%|          | 0/20 [00:00<?, ?it/s]

text_id,page_id,name,onset,offset,duration,peak_velocity,amplitude,dispersion
i64,i64,str,i64,i64,i64,f64,f64,f64
0,1,"""fixation""",1988145,1988563,418,200.144558,2.492864,2.712569
0,1,"""fixation""",1988564,1988750,186,249.67823,2.651198,2.865026
0,1,"""fixation""",1988751,1989178,427,211.598748,2.585906,2.779518
0,1,"""fixation""",1989179,1989436,257,189.183243,2.614347,2.77424
0,1,"""fixation""",1989437,1989600,163,255.077509,2.594651,2.729391
…,…,…,…,…,…,…,…,…
0,1,"""saccade""",2004373,2004385,12,70.374183,0.7073,0.766684
0,1,"""saccade""",2004688,2004704,16,175.646379,1.807485,1.875716
0,1,"""saccade""",2004879,2004901,22,209.46361,2.933818,3.086169


This way we can compute all of our desired properties in a single run.

## Saving Event Data

Saving your event data is as simple as:

In [32]:
dataset.save_events()

  0%|          | 0/20 [00:00<?, ?it/s]

text_id,page_id,name,onset,offset,duration,peak_velocity,amplitude,dispersion
i64,i64,str,i64,i64,i64,f64,f64,f64
0,1,"""fixation""",1988145,1988563,418,200.144558,2.492864,2.712569
0,1,"""fixation""",1988564,1988750,186,249.67823,2.651198,2.865026
0,1,"""fixation""",1988751,1989178,427,211.598748,2.585906,2.779518
0,1,"""fixation""",1989179,1989436,257,189.183243,2.614347,2.77424
0,1,"""fixation""",1989437,1989600,163,255.077509,2.594651,2.729391
…,…,…,…,…,…,…,…,…
0,1,"""saccade""",2004373,2004385,12,70.374183,0.7073,0.766684
0,1,"""saccade""",2004688,2004704,16,175.646379,1.807485,1.875716
0,1,"""saccade""",2004879,2004901,22,209.46361,2.933818,3.086169


All of the event data is saved into this directory:

In [33]:
dataset.paths.events

WindowsPath('data/ToyDataset/events')

Let's confirm it by printing all files in this directory:

In [34]:
print(list(dataset.paths.events.glob('*/*/*')))

[WindowsPath('data/ToyDataset/events/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_0_1.feather'), WindowsPath('data/ToyDataset/events/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_0_2.feather'), WindowsPath('data/ToyDataset/events/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_0_3.feather'), WindowsPath('data/ToyDataset/events/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_0_4.feather'), WindowsPath('data/ToyDataset/events/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_0_5.feather'), WindowsPath('data/ToyDataset/events/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_1_1.feather'), WindowsPath('data/ToyDataset/events/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_1_2.feather'), WindowsPath('data/ToyDataset/events/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_1_3.feather'), WindowsPath('data/ToyDataset/events/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_1_4.feather'), WindowsPath('data/ToyDataset/events/aeye-lab-pymovements-toy-dataset-6cb

All files have been saved into the `Dataset.paths.events` as files in [Feather format](https://arrow.apache.org/docs/python/feather.html). 

If we want to save the data into an alternative directory and also use a different file format like `csv` we can use the following:

In [35]:
dataset.save_events(events_dirname='events_csv', extension='csv')

  0%|          | 0/20 [00:00<?, ?it/s]

text_id,page_id,name,onset,offset,duration,peak_velocity,amplitude,dispersion
i64,i64,str,i64,i64,i64,f64,f64,f64
0,1,"""fixation""",1988145,1988563,418,200.144558,2.492864,2.712569
0,1,"""fixation""",1988564,1988750,186,249.67823,2.651198,2.865026
0,1,"""fixation""",1988751,1989178,427,211.598748,2.585906,2.779518
0,1,"""fixation""",1989179,1989436,257,189.183243,2.614347,2.77424
0,1,"""fixation""",1989437,1989600,163,255.077509,2.594651,2.729391
…,…,…,…,…,…,…,…,…
0,1,"""saccade""",2004373,2004385,12,70.374183,0.7073,0.766684
0,1,"""saccade""",2004688,2004704,16,175.646379,1.807485,1.875716
0,1,"""saccade""",2004879,2004901,22,209.46361,2.933818,3.086169


Let's confirm again by printing all the new files in this alternative directory:

In [36]:
alternative_dirpath = dataset.path / 'events_csv'
print(list(alternative_dirpath.glob('*/*/*')))

[WindowsPath('data/ToyDataset/events_csv/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_0_1.csv'), WindowsPath('data/ToyDataset/events_csv/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_0_2.csv'), WindowsPath('data/ToyDataset/events_csv/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_0_3.csv'), WindowsPath('data/ToyDataset/events_csv/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_0_4.csv'), WindowsPath('data/ToyDataset/events_csv/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_0_5.csv'), WindowsPath('data/ToyDataset/events_csv/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_1_1.csv'), WindowsPath('data/ToyDataset/events_csv/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_1_2.csv'), WindowsPath('data/ToyDataset/events_csv/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_1_3.csv'), WindowsPath('data/ToyDataset/events_csv/aeye-lab-pymovements-toy-dataset-6cb5d66/data/trial_1_4.csv'), WindowsPath('data/ToyDataset/events_csv/aeye-lab-pymovements-toy-dataset

### Loading Previously Computed Events Data

Let's initialize a new dataset object from the same `ToyDataset`.

In [41]:
preprocessed_dataset = pm.Dataset('ToyDataset', path='data/ToyDataset')

When we load the dataset using `load()` without any parameters there will be no events loaded:

In [43]:
preprocessed_dataset.load()

  0%|          | 0/20 [00:00<?, ?it/s]

text_id,page_id,filepath
i64,i64,str
0,1,"""aeye-lab-pymovements-toy-datas…"
0,2,"""aeye-lab-pymovements-toy-datas…"
0,3,"""aeye-lab-pymovements-toy-datas…"
0,4,"""aeye-lab-pymovements-toy-datas…"
0,5,"""aeye-lab-pymovements-toy-datas…"
…,…,…
3,1,"""aeye-lab-pymovements-toy-datas…"
3,2,"""aeye-lab-pymovements-toy-datas…"
3,3,"""aeye-lab-pymovements-toy-datas…"


But when we load it with the `events=True` parameter the events will be loaded:


In [44]:
preprocessed_dataset.load(events=True)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

text_id,page_id,name,onset,offset,duration,peak_velocity,amplitude,dispersion
i64,i64,str,i64,i64,i64,f64,f64,f64
0,1,"""fixation""",1988145,1988563,418,200.144558,2.492864,2.712569
0,1,"""fixation""",1988564,1988750,186,249.67823,2.651198,2.865026
0,1,"""fixation""",1988751,1989178,427,211.598748,2.585906,2.779518
0,1,"""fixation""",1989179,1989436,257,189.183243,2.614347,2.77424
0,1,"""fixation""",1989437,1989600,163,255.077509,2.594651,2.729391
…,…,…,…,…,…,…,…,…
0,1,"""saccade""",2004373,2004385,12,70.374183,0.7073,0.766684
0,1,"""saccade""",2004688,2004704,16,175.646379,1.807485,1.875716
0,1,"""saccade""",2004879,2004901,22,209.46361,2.933818,3.086169


By default, the `events` directory and the `feather` extension will be chosen.

In case of alternative directory names or other file formats you can use the following:

In [45]:
preprocessed_dataset.load(
    events=True,
    events_dirname='events_csv',
    extension='csv',
)
dataset.events[0]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

text_id,page_id,name,onset,offset,duration,peak_velocity,amplitude,dispersion
i64,i64,str,i64,i64,i64,f64,f64,f64
0,1,"""fixation""",1988145,1988563,418,200.144558,2.492864,2.712569
0,1,"""fixation""",1988564,1988750,186,249.67823,2.651198,2.865026
0,1,"""fixation""",1988751,1989178,427,211.598748,2.585906,2.779518
0,1,"""fixation""",1989179,1989436,257,189.183243,2.614347,2.77424
0,1,"""fixation""",1989437,1989600,163,255.077509,2.594651,2.729391
…,…,…,…,…,…,…,…,…
0,1,"""saccade""",2004373,2004385,12,70.374183,0.7073,0.766684
0,1,"""saccade""",2004688,2004704,16,175.646379,1.807485,1.875716
0,1,"""saccade""",2004879,2004901,22,209.46361,2.933818,3.086169


## What you have learned in this tutorial:

* detecting different events with different algorithms by using `Dataset.detect_events()`
* computing event properties by using `Dataset.compute_event_properties()`
* saving your preprocesed data using `Dataset.save_preprocessed()`
* load your preprocesed data using `Dataset.load(events=True)`
* using custom directory names by specifying `preprocessed_dirname`
* using other file formats than the default `feather` format by specifying `extension`